## Introduction

Every year, Formula One travels all around the world to host races in different countries. Huge numbers of people and lots of equipment have to be hauled from location to location, causing significant environmental impact.

Formula One needs to do everything it can to lessen its carbon emissions, given that it's a spectacle that entirely revolves around motor cars. Some improvements have been made, such as the move to hybrid engines in 2014, but there's a lot more that could be done. 

One particularly simple change would be to organise the season so that the total distance travelled between race weekends is minimised. As I'll show, the current calendar order is very inefficient and involves back-and-forth trips between continents. I will propose an improved ordering with a lower total distance travelled, and hence improved carbon emissions.

It should be noted that this analysis will be based on straight line distances from racetrack to racetrack, and assumes that the same number of people/amount of equipment is being hauled each time. In reality, I imagine that there's a lot of variation in what is hauled, where it's hauled via, and how it's hauled, so in that regard this will be a fairly simplistic exercise. I nonetheless expect it to be indicative of the kind of improvements F1 could make.

### Loading and explorting the data

I'll be using the handy [Ergast API](http://ergast.com/mrd/) to get a season's schedule with longitudes and latitudes.

I'll then visualise these journeys using Plotly's mapping functionality.